# Astronomical Coordinates 1: Getting Started with astropy.coordinates

## Authors
Adrian Price-Whelan

## Learning Goals
* Create `astropy.coordinates.SkyCoord` objects using names and coordinates
* Use SkyCoord objects to become familiar with object oriented programming (OOP)
* Interact with a `SkyCoord` object and access its attributes
* Use a `SkyCoord` object to query a database

## Keywords
coordinates, OOP, astroquery


## Summary

Astronomers use a wide variety of coordinate systems and formats to represent sky coordinates of celestial objects. For example, you may be familiar with terms like "right ascension" and "declination" or "galactic latitude and longitude," and you may have seen angular coordinate components represented as "0h39m15.9s," "00:39:15.9," or 9.81625º. `astropy.coordinates` provides tools for representing the coordinates of objects and transforming them between different systems. 

In this tutorial, we will explore how the `astropy.coordinates` package can be used to work with astronomical coordinates. You may find it helpful to keep [the Astropy documentation for the coordinates package](http://docs.astropy.org/en/stable/coordinates/index.html) open alongside this tutorial for reference or additional reading. In the text below, you may also see some links that look like ([docs](http://docs.astropy.org/en/stable/coordinates/index.html)). These links will take you to parts of the documentation that are directly relevant to the cells from which they link. 

## Imports

We start by importing some general packages we will need below:

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from astropy import units as u
from astropy.coordinates import SkyCoord, Distance
from astropy.io import fits
from astropy.table import QTable
from astropy.utils.data import download_file

## Representing on-sky locations with `astropy.coordinates`

In Astropy, the most common way of representing and working with sky coordinates is to use the `SkyCoord` object ([docs](https://docs.astropy.org/en/stable/coordinates/skycoord.html)). A `SkyCoord` can be created directly from angles or arrays of angles with associated units, as demonstrated below. 

To get started, let's assume that we want to create a `SkyCoord` object for the center of the open cluster NGC 188 so that later we can query and retrieve stars that might be members of the cluster. Let's also assume, for now, that we already know the sky coordinates of the cluster to be (12.11, 85.26) degrees in the ICRS coordinate frame. The ICRS — sometimes referred to as "equitorial" or "J2000" coordinates — is currently the most common astronomical coordinate frame for stellar or extragalactic astronomy, and is the default coordinate frame for `SkyCoord`. We could therefore create a `SkyCoord` object for the center of NGC 188 as:

In [2]:
ngc188_center = SkyCoord(12.11*u.deg, 85.26*u.deg)
ngc188_center

<SkyCoord (ICRS): (ra, dec) in deg
    (12.11, 85.26)>

It is generally recommended to explicitly specify the frame your coordinates are in. In this case, because ICRS is the default, this would be an equivalent way of creating our `SkyCoord` object:

In [3]:
ngc188_center = SkyCoord(12.11*u.deg, 85.26*u.deg, frame='icrs')
ngc188_center

<SkyCoord (ICRS): (ra, dec) in deg
    (12.11, 85.26)>

As we will see below, there are many other supported coordinate frames.

In the above initializations, we passed in `astropy.units.Quantity` objects with angular units to specify the angular components of our sky coordinates. However, `SkyCoord` will also accept string-formatted coordinates either as separate strings for Right Ascension (RA) and Declination (Dec) or a single string:

In [4]:
SkyCoord('00h48m26.4s', '85d15m36s', frame='icrs')

<SkyCoord (ICRS): (ra, dec) in deg
    (12.11, 85.26)>

Some string representations do not explicitly define units, so you would have to specify the units explicitly if they are not included in the string:

In [5]:
SkyCoord('00:48:26.4 85:15:36', unit=(u.hour, u.deg), 
         frame='icrs')

<SkyCoord (ICRS): (ra, dec) in deg
    (12.11, 85.26)>

For more information and examples on initializing `SkyCoord` objects, [see this documentation](http://docs.astropy.org/en/latest/coordinates/skycoord.html).

For the initializations above, we assumed that we already had the coordinate component values ready. If you do not know the coordinate values, and the object you are interested in is in [SESAME](http://cdsweb.u-strasbg.fr/cgi-bin/Sesame), you can also automatically look up and load coordinate values from the name of the object using the `SkyCoord.from_name()` class method<sup>1</sup> ([docs](http://docs.astropy.org/en/latest/coordinates/index.html#convenience-methods)). Note, however, that this requires an internet connection. It is safe to skip this cell if you are not connected to the internet because we already defined it above.

<sub> <sup>1</sup>If you don't know what a class method is, think of it like an alternative constructor for a `SkyCoord` object — calling `SkyCoord.from_name()` with a name gives you a new `SkyCoord` object. For more detailed background on what class methods are and when they're useful, see [this page](https://julien.danjou.info/blog/2013/guide-python-static-class-abstract-methods).</sub>

In [6]:
ngc188_center = SkyCoord.from_name('NGC 188')
ngc188_center

<SkyCoord (ICRS): (ra, dec) in deg
    (12.10833333, 85.255)>

The `SkyCoord` object we created has various ways of accessing the information contained within it. All `SkyCoord` objects have attributes that allow you to retrieve the coordinate component data, but the component names will change depending on the coordinate frame of the `SkyCoord` you have. In our case, we have a `SkyCoord` with ICRS coordinates. The component names are therefore lower-case abbreviations of Right Ascension, `.ra`, and Declination, `.dec`:

In [7]:
ngc188_center.ra, ngc188_center.dec

(<Longitude 12.10833333 deg>, <Latitude 85.255 deg>)

These ``ra`` and ``dec`` attributes return specialized [``Quantity``](http://docs.astropy.org/en/stable/units/index.html) objects (actually, a subclass called [``Angle``](http://docs.astropy.org/en/stable/api/astropy.coordinates.Angle.html), which in turn is subclassed by [``Latitude``](http://docs.astropy.org/en/stable/api/astropy.coordinates.Latitude.html) and [``Longitude``](http://docs.astropy.org/en/stable/api/astropy.coordinates.Longitude.html)).  These objects store angles and provide pretty representations of those angles, as well as some useful attributes to quickly convert to common angle units. For example, in a Jupyter notebook, these objects know how to represent themselves using LaTeX:

In [8]:
ngc188_center.ra

<Longitude 12.10833333 deg>

In [9]:
ngc188_center.dec

<Latitude 85.255 deg>

We can also retrieve the component values in different units:

In [10]:
ngc188_center.ra.hour, ngc188_center.ra.radian, ngc188_center.ra.degree

(0.8072222220000002, 0.2113302835374691, 12.10833333)

And format the values into strings with specified units ([docs](http://docs.astropy.org/en/latest/coordinates/formatting.html)), for example:

In [11]:
ngc188_center.ra.to_string(unit=u.hourangle, sep=':')

'0:48:26'

Now that we have a `SkyCoord` object for the center of NGC 188, we can select sources from the *Gaia* Data Release 2 catalog around this position to try to find stars that might be members of the cluster. To do this, we will use the `astroquery.gaia` module to query the *Gaia* data archive ([docs](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html)).

This requires an internet connection, but if it fails, the catalog file is included in the repository so you can load it locally (i.e., skip the next cell if you do not have an internet connection):

In [12]:
from astroquery.gaia import Gaia

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [13]:
job = Gaia.cone_search_async(ngc188_center, radius=0.5*u.deg)
table = job.get_results()

# only keep stars brighter than G=19 magnitude
table = table[table['phot_g_mean_mag'] < 19*u.mag]

INFO: Query finished. [astroquery.utils.tap.core]


The above cell may not work if you do not have an internet connection, so we have included the results table along with the notebook:

In [15]:
table = QTable.read('gaia_results.fits')

In [16]:
len(table)

4977

This table now contains about 5000 stars from *Gaia* DR2 around the coordinate position of the center of NGC 188. Let's now construct a `SkyCoord` object with the results table. In the *Gaia* data archive, the ICRS coordinates of a source are given as column names `"ra"` and `"dec"`:

In [17]:
table['ra']

<Quantity [12.12843989, 12.09381253, 12.09832966, ..., 14.15160862,
           12.6107374 ,  9.634002  ] deg>

In [18]:
table['dec']

<Quantity [85.26076889, 85.26522648, 85.25443128, ..., 84.79281259,
           85.75826293, 84.80832667] deg>

Note that, because the *Gaia* archive provides data tables with associated units, and we read this table with the `QTable` object ([docs](http://docs.astropy.org/en/latest/table/mixin_columns.html#quantity-and-qtable)), the above columns are represented as `Quantity` objects with units of degrees. Note also that these columns contain many (>5000!) coordinate values. We can pass these directly in to `SkyCoord` to get a single `SkyCoord` object to represent all of these coordinates:

In [19]:
gaia_coords = SkyCoord(table['ra'], table['dec'])
gaia_coords

<SkyCoord (ICRS): (ra, dec) in deg
    [(12.12843989, 85.26076889), (12.09381253, 85.26522648),
     (12.09832966, 85.25443128), ..., (14.15160862, 84.79281259),
     (12.6107374 , 85.75826293), ( 9.634002  , 84.80832667)]>

### Exercises

Create a `SkyCoord` of the open cluster the Pleiades (either by looking up the coordinates and passing them in, or by using the convenience method we learned about above):

In [20]:
pleiades_center = SkyCoord.from_name('Pleiades')

Using only a single method/function call on the `SkyCoord` object representing the center of NGC 188, print a string with the RA/Dec in the form 'HH:MM:SS.S DD:MM:SS.S'. Check your answer against [SIMBAD](http://simbad.u-strasbg.fr/simbad/), which will show you sexagesimal coordinates for the object.

<sub>(Hint: `SkyCoord.to_string()` might be useful.)</sub>

In [21]:
ngc188_center.to_string(style="hmsdms", sep=":", precision=1)

'00:48:26.0 +85:15:18.0'

Using a single method/function call on the `SkyCoord` object containing the results of our *Gaia* query, compute the angular separation between each resulting star and the coordinates of the cluster center.

<sub>(Hint: `SkyCoord.separation()` might be useful.)</sub>

In [22]:
gaia_coords.separation(ngc188_center)

<Angle [0.00600359, 0.01029663, 0.00100414, ..., 0.49493249, 0.50479495,
        0.49531568] deg>

---

## More than just sky positions: Including distance information in `SkyCoord`

So far, we have only used `SkyCoord` to represent sky positions (i.e., `ra` and `dec` only). It is sometimes useful to include distance information with the sky coordinates, thereby fully specifying the 3D position of a given source. To pass in distance information, `SkyCoord` accepts a keyword argument `distance`. So, if we knew the distance to NGC 188, we could also pass in a distance (as a `Quantity` object) using this argument:

In [23]:
ngc188_center_3d = SkyCoord(12.11*u.deg, 85.26*u.deg,
                            distance=1.96*u.kpc)

With our results from querying the *Gaia* catalog, we are also given parallax measurements to each star in the catalog. The default way of passing in a distance, as above, is to pass in a `Quantity` with a unit of length. However, `astropy.coordinates` also provides a specialized object, `Distance`, for handling common transformations of different distance representations ([docs](http://docs.astropy.org/en/latest/coordinates/index.html#distance)). Among other things, this class supports passing in a parallax value (but note that the transformation to distance here inverts the parallax, which is not always a good idea!):

In [24]:
Distance(parallax=1*u.mas)

<Distance 1000. pc>

The catalog of stars we queried from *Gaia* contains parallax information, so we can also create a `SkyCoord` object to represent the 3D positions of all of the *Gaia* stars. Here, we sub-select the table to return only stars with positive parallax values:

In [25]:
table_3d = table[table['parallax'] > 0]

gaia_dist = Distance(parallax=table_3d['parallax'])
gaia_coords_3d = SkyCoord(table_3d['ra'], table_3d['dec'],
                          distance=gaia_dist)
gaia_coords_3d

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    [(12.12843989, 85.26076889, 1835.66340989),
     (12.09381253, 85.26522648, 2184.90012993),
     (12.09832966, 85.25443128, 1109.14781668), ...,
     (14.15160862, 84.79281259, 1159.84996432),
     (12.6107374 , 85.75826293, 2412.31892   ),
     ( 9.634002  , 84.80832667, 1040.65423705)]>

Now that we have 3D position information for both the cluster center, and for the stars we queried from *Gaia*, we can compute the 3D separation (distance) between all of the *Gaia* sources and the cluster center:

In [26]:
sep3d = gaia_coords_3d.separation_3d(ngc188_center_3d)
sep3d

<Distance [124.33660295, 224.90021433, 850.85219575, ..., 800.25800151,
           452.71644735, 919.43018437] pc>

### Exercises

Using the 3D separation values, define a boolean mask to select candidate members of the cluster. Select all stars within 50 pc of the cluster center. How many candidate members of NGC 188 do we have, based on their 3D positions?

In [27]:
ngc188_3d_mask = sep3d < 50*u.pc
ngc188_3d_mask.sum()

321

In this tutorial, we have introduced `astropy.coordinates` as a way to store and represent astronomical sky coordinates. We used coordinate objects, via the `SkyCoord` class interface, to parse and change coordinate representations and units. We also demonstrated how to TODO